Connexion SQLite

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/processed/airbnb_analysis.db")


1) KPI — Nombre d’annonces par ville (via schéma étoile)

In [2]:
query = """
SELECT c.city AS city,
       COUNT(*) AS nb_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
GROUP BY c.city
ORDER BY nb_listings DESC;
"""
pd.read_sql(query, conn)


,city,nb_listings
0,Madrid,18833
1,Barcelona,15054


2) KPI — Prix moyen par ville

In [3]:
query = """
SELECT c.city AS city,
       ROUND(AVG(f.price), 2) AS avg_price
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
GROUP BY c.city
ORDER BY avg_price DESC;
"""
pd.read_sql(query, conn)


,city,avg_price
0,Barcelona,158.26
1,Madrid,134.16


3) KPI — Prix médian par ville (SQL pur)

In [4]:
query = """
WITH ranked AS (
  SELECT
    c.city AS city,
    f.price AS price,
    ROW_NUMBER() OVER (PARTITION BY c.city ORDER BY f.price) AS rn,
    COUNT(*) OVER (PARTITION BY c.city) AS cnt
  FROM fact_listings f
  JOIN dim_city c ON f.city_id = c.city_id
)
SELECT city,
       ROUND(AVG(price), 2) AS median_price
FROM ranked
WHERE rn IN ((cnt + 1) / 2, (cnt + 2) / 2)
GROUP BY city;
"""
pd.read_sql(query, conn)


,city,median_price
0,Barcelona,129.0
1,Madrid,110.0


4) KPI — Prix moyen par type de logement et par ville

In [5]:
query = """
SELECT c.city AS city,
       rt.room_type AS room_type,
       ROUND(AVG(f.price), 2) AS avg_price,
       COUNT(*) AS nb_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
JOIN dim_room_type rt ON f.room_type_id = rt.room_type_id
GROUP BY c.city, rt.room_type
ORDER BY c.city, avg_price DESC;
"""
pd.read_sql(query, conn)


,city,room_type,avg_price,nb_listings
0,Barcelona,Hotel room,223.94,50
1,Barcelona,Entire home/apt,191.60,10288
2,Barcelona,Private room,85.01,4610
3,Barcelona,Shared room,76.21,106
4,Madrid,Entire home/apt,157.63,13561
5,Madrid,Hotel room,151.10,41
6,Madrid,Private room,73.94,5084
7,Madrid,Shared room,46.42,147


5) KPI — Disponibilité moyenne par ville

In [6]:
query = """
SELECT c.city AS city,
       ROUND(AVG(f.availability_365), 2) AS avg_availability
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
GROUP BY c.city
ORDER BY avg_availability DESC;
"""
pd.read_sql(query, conn)


,city,avg_availability
0,Barcelona,227.26
1,Madrid,212.92


6) KPI — Top quartiers les plus chers (avec seuil pour éviter bruit)

In [7]:
query = """
SELECT c.city AS city,
       n.neighbourhood AS neighbourhood,
       ROUND(AVG(f.price), 2) AS avg_price,
       COUNT(*) AS nb_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
JOIN dim_neighbourhood n ON f.neighbourhood_id = n.neighbourhood_id
GROUP BY c.city, n.neighbourhood
HAVING nb_listings >= 30
ORDER BY avg_price DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,city,neighbourhood,avg_price,nb_listings
0,Barcelona,Diagonal Mar i el Front Marítim del Poblenou,241.86,126
1,Barcelona,la Dreta de l'Eixample,223.32,1902
2,Madrid,Recoletos,216.59,260
3,Barcelona,la Vila Olímpica del Poblenou,213.86,132
4,Madrid,Castellana,208.83,160
5,Madrid,Goya,187.02,313
6,Barcelona,Sant Antoni,185.49,794
7,Barcelona,l'Antiga Esquerra de l'Eixample,185.34,787
8,Barcelona,el Fort Pienc,178.73,385
9,Madrid,Cortes,176.94,855


In [8]:
conn.close()
print("✅ Étape 2 terminée : KPI SQL prêts.")


✅ Étape 2 terminée : KPI SQL prêts.


vérifier les tables + colonnes (sécurise les noms)

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/processed/airbnb_analysis.db")

# Tables dispo
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,listings
1,dim_city
2,dim_room_type
3,dim_neighbourhood
4,fact_listings


In [2]:
pd.read_sql("PRAGMA table_info(fact_listings);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,listing_id,INTEGER,0,None,0
1,1,city_id,INTEGER,0,None,0
2,2,room_type_id,INTEGER,0,None,0
3,3,neighbourhood_id,INTEGER,0,None,0
4,4,price,REAL,0,None,0
5,5,availability_365,INTEGER,0,None,0
6,6,minimum_nights,INTEGER,0,None,0
7,7,number_of_reviews,INTEGER,0,None,0
8,8,latitude,REAL,0,None,0
9,9,longitude,REAL,0,None,0


fonction pour exécuter SQL

In [3]:
def q(sql: str):
    return pd.read_sql(sql, conn)


KPI 1 — Prix moyen / médian par ville

In [4]:
kpi1 = q("""
SELECT
  c.city,
  ROUND(AVG(f.price), 2) AS avg_price,
  ROUND(MIN(f.price), 2) AS min_price,
  ROUND(MAX(f.price), 2) AS max_price,
  COUNT(*) AS n_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
GROUP BY c.city
ORDER BY avg_price DESC;
""")
kpi1


,city,avg_price,min_price,max_price,n_listings
0,Barcelona,158.26,9.0,974.0,15054
1,Madrid,134.16,8.0,971.0,18833


KPI 2 — Prix moyen par ville et type de logement

In [5]:
kpi2 = q("""
SELECT
  c.city,
  r.room_type,
  ROUND(AVG(f.price), 2) AS avg_price,
  COUNT(*) AS n_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
JOIN dim_room_type r ON f.room_type_id = r.room_type_id
GROUP BY c.city, r.room_type
ORDER BY avg_price DESC;
""")
kpi2


,city,room_type,avg_price,n_listings
0,Barcelona,Hotel room,223.94,50
1,Barcelona,Entire home/apt,191.60,10288
2,Madrid,Entire home/apt,157.63,13561
3,Madrid,Hotel room,151.10,41
4,Barcelona,Private room,85.01,4610
5,Barcelona,Shared room,76.21,106
6,Madrid,Private room,73.94,5084
7,Madrid,Shared room,46.42,147


KPI 3 — Disponibilité moyenne (availability_365) par ville

In [6]:
kpi3 = q("""
SELECT
  c.city,
  ROUND(AVG(f.availability_365), 2) AS avg_availability_365,
  ROUND(MIN(f.availability_365), 2) AS min_availability_365,
  ROUND(MAX(f.availability_365), 2) AS max_availability_365,
  COUNT(*) AS n_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
GROUP BY c.city
ORDER BY avg_availability_365 DESC;
""")
kpi3


,city,avg_availability_365,min_availability_365,max_availability_365,n_listings
0,Barcelona,227.26,0.0,365.0,15054
1,Madrid,212.92,0.0,365.0,18833


KPI 4 — Top quartiers les plus chers

In [7]:
q("SELECT name FROM sqlite_master WHERE type='table' AND name='dim_neighbourhood';")


,name
0,dim_neighbourhood


In [8]:
kpi4 = q("""
SELECT
  c.city,
  n.neighbourhood,
  ROUND(AVG(f.price), 2) AS avg_price,
  COUNT(*) AS n_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
JOIN dim_neighbourhood n ON f.neighbourhood_id = n.neighbourhood_id
GROUP BY c.city, n.neighbourhood
HAVING n_listings >= 30
ORDER BY avg_price DESC
LIMIT 20;
""")
kpi4


,city,neighbourhood,avg_price,n_listings
0,Barcelona,Diagonal Mar i el Front Marítim del Poblenou,241.86,126
1,Barcelona,la Dreta de l'Eixample,223.32,1902
2,Madrid,Recoletos,216.59,260
3,Barcelona,la Vila Olímpica del Poblenou,213.86,132
4,Madrid,Castellana,208.83,160
5,Madrid,Goya,187.02,313
6,Barcelona,Sant Antoni,185.49,794
7,Barcelona,l'Antiga Esquerra de l'Eixample,185.34,787
8,Barcelona,el Fort Pienc,178.73,385
9,Madrid,Cortes,176.94,855


KPI 5 — Relation prix vs disponibilité (agrégé par ville)

In [9]:
kpi5 = q("""
SELECT
  c.city,
  ROUND(AVG(f.price), 2) AS avg_price,
  ROUND(AVG(f.availability_365), 2) AS avg_availability_365,
  COUNT(*) AS n_listings
FROM fact_listings f
JOIN dim_city c ON f.city_id = c.city_id
GROUP BY c.city;
""")
kpi5


,city,avg_price,avg_availability_365,n_listings
0,Barcelona,158.26,227.26,15054
1,Madrid,134.16,212.92,18833


In [10]:
kpi5_corr = kpi5[["avg_price", "avg_availability_365"]].corr()
kpi5_corr


,avg_price,avg_availability_365
avg_price,1.0,1.0
avg_availability_365,1.0,1.0


Export KPI → CSV

In [11]:
# On exporte en format "long" simple pour BI (une table par KPI)
kpi1.to_csv("../data/processed/kpi1_price_by_city.csv", index=False)
kpi2.to_csv("../data/processed/kpi2_price_by_city_room_type.csv", index=False)

# Ceux-ci existent seulement si calculés
try:
    kpi3.to_csv("../data/processed/kpi3_availability_by_city.csv", index=False)
except:
    pass

try:
    kpi4.to_csv("../data/processed/kpi4_top_neighbourhoods.csv", index=False)
except:
    pass

kpi5.to_csv("../data/processed/kpi5_price_vs_availability.csv", index=False)

print("✅ KPI exportés dans data/processed/")


✅ KPI exportés dans data/processed/


In [12]:
conn.close()
print("Connexion SQLite fermée.")


Connexion SQLite fermée.
